In [0]:
train_df = spark.read.option("header", True).option("inferSchema", True)\
    .csv("/Volumes/workspace/default/santander_dataset/train.csv")

test_df = spark.read.option("header", True).option("inferSchema", True)\
    .csv("/Volumes/workspace/default/santander_dataset/test.csv")

train_df.show(5, truncate=False)
test_df.show(5, truncate=False)

+-------+------+-------+-------+-------+------+-------+-------+------+-------+-------+------+-------+--------+-------+-------+------+-------+-------+--------+-------+-------+-------+-------+-------+------+-------+-------+--------+-------+------+------+--------+-------+-------+-------+-------+-------+------+------+-------+------+--------+-------+-------+-------+-------+--------+-------+--------+-------+-------+-------+-------+-------+------+-------+-------+-------+------+------+-------+-------+--------+-------+-------+------+-------+------+-------+------+-------+-------+------+-------+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+-------+-------+--------+-------+-------+-------+--------+------+-------+-------+-------+-------+-------+-------+------+-------+--------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+-------+-------+-------+-------+-------+

In [0]:
from pyspark.sql.functions import lit

test_df = test_df.withColumn("target", lit(None))

combined_df = train_df.unionByName(test_df)

combined_df.show(10, truncate=False)



# ОЗНАЙОМЛЕННЯ ЗІ СТРУКТРОЮ ДАНИХ

print("\nРозмір датасету:")
print("Кількість рядків:", combined_df.count())
print("Кількість колонок:", len(combined_df.columns))

print("Схема датасету:")
combined_df.printSchema()


+-------+------+-------+-------+-------+------+-------+--------+------+-------+-------+------+-------+--------+-------+-------+-------+-------+-------+--------+-------+-------+-------+-------+-------+------+-------+-------+--------+-------+------+-------+--------+-------+-------+-------+-------+-------+------+------+-------+-------+--------+-------+-------+-------+-------+--------+-------+--------+-------+-------+-------+-------+-------+------+--------+-------+-------+------+-------+-------+-------+--------+-------+-------+------+-------+------+-------+------+-------+-------+------+-------+-------+-------+-------+-------+-------+------+-------+-------+-------+--------+-------+-------+-------+--------+-------+-------+-------+--------+------+-------+-------+-------+-------+-------+--------+------+-------+--------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+--------+-------+-------+-------+-------

In [0]:

from pyspark.sql.functions import col, count, when, min, max, mean, countDistinct, lit, expr

# Перетворюємо всі var_0…var_199 у числові колонки
numeric_cols = [f"var_{i}" for i in range(200)]
for c in numeric_cols:
    combined_df = combined_df.withColumn(c, col(c).cast("double"))

# Пропуски у всіх колонках
print("Пропуски у всіх колонках:")
combined_df.select([count(when(col(c).isNull(), c)).alias(c) for c in combined_df.columns]).show(truncate=False)

# Кількість дублікатів
duplicate_count = combined_df.count() - combined_df.dropDuplicates().count()
print("Кількість дублікатів:", duplicate_count)

# Статистика числових колонок (мін, макс, середнє, медіана)
print("Статистика числових колонок:")
median_exprs = [expr(f'percentile_approx({c}, 0.5)').alias(f'{c}_median') for c in numeric_cols]
combined_df.select(
    *[min(c).alias(f"{c}_min") for c in numeric_cols],
    *[max(c).alias(f"{c}_max") for c in numeric_cols],
    *[mean(c).alias(f"{c}_mean") for c in numeric_cols],
    *median_exprs
).show(truncate=False)

# Аналіз категоріальної змінної target
print("Категоріальна колонка 'target':")
combined_df.select(countDistinct("target")).show()
combined_df.groupBy("target").count().show()



Пропуски у всіх колонках:
+-------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+----

In [0]:
from pyspark.sql.functions import col, mean, when, count

# Видалення дублікатів
combined_cleaned = combined_df.dropDuplicates()
print("Було рядків:", combined_df.count())
print("Після видалення дублікатів:", combined_cleaned.count())

# Список числових колонок
numeric_cols = [f"var_{i}" for i in range(200)]

# Обчислення середніх значень для кожної числової колонки
mean_values = combined_cleaned.select(
    *[mean(col(c)).alias(c) for c in numeric_cols]
).first().asDict()

# Заповнення пропусків середніми значеннями
combined_cleaned = combined_cleaned.na.fill(mean_values)

# Усунення від’ємних значень (якщо є)
for c in numeric_cols:
    combined_cleaned = combined_cleaned.withColumn(c, when(col(c) < 0, 0).otherwise(col(c)))

# Перевірка якості очищення
print("Перевірка пропусків після очищення:")
combined_cleaned.select([
    count(when(col(c).isNull(), c)).alias(c) for c in combined_cleaned.columns
]).show(truncate=False)

print("Приклад перших рядків:")
combined_cleaned.select(["ID_code", "target"] + numeric_cols[:5]).show(5, truncate=False)


Було рядків: 400000
Після видалення дублікатів: 400000
Перевірка пропусків після очищення:
+-------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+---

In [0]:
# СТАНДАРТИЗАЦІЯ ТА ТРАНСФОРМАЦІЯ

from pyspark.sql.functions import col, mean, stddev, when

# Стандартизація числових ознак

numeric_cols = [f"var_{i}" for i in range(200)]

# Середнє і стандартне відхилення для кожної колонки
stats = combined_cleaned.select([
    mean(col(c)).alias(f"{c}_mean") for c in numeric_cols
] + [
    stddev(col(c)).alias(f"{c}_std") for c in numeric_cols
]).collect()[0]

# Новий датафрейм зі стандартизованими ознаками
combined_standardized = combined_cleaned
for c in numeric_cols:
    mean_val = stats[f"{c}_mean"]
    std_val = stats[f"{c}_std"] if stats[f"{c}_std"] != 0 and stats[f"{c}_std"] is not None else 1
    combined_standardized = combined_standardized.withColumn(
        c, (col(c) - mean_val) / std_val
    )

print(" Стандартизація виконана успішно")
print("Перевірка результату:")
combined_standardized.select(numeric_cols[:5]).show(5, truncate=False)


 Стандартизація виконана успішно
Перевірка результату:
+-------------------+--------------------+--------------------+---------------------+--------------------+
|var_0              |var_1               |var_2               |var_3                |var_4               |
+-------------------+--------------------+--------------------+---------------------+--------------------+
|2.147899518026267  |-0.5524378631346035 |-0.7265211426199911 |0.8776387536726915   |-2.4750924979459885 |
|0.32065391470018184|-0.42981866225842125|1.589439943111964   |-0.27864465101049907 |-1.365143796701357  |
|-1.0983173733597764|-0.5524378631346035 |-0.24964875062819675|-1.1751170681374525  |0.3606816631848261  |
|-0.681650478456982 |-0.5524378631346035 |-0.7590153495005639 |0.9247084383192261   |-0.1558003900223933 |
|1.1070221706174384 |-0.5524378631346035 |0.7246087123780182  |-0.020591452507451733|-0.32390708747437214|
+-------------------+--------------------+--------------------+---------------------+----

In [0]:
# ЗБЕРЕЖЕННЯ РЕЗУЛЬТАТУ
delta_path = "/Volumes/workspace/default/santader_delta_dataset"

combined_standardized.write.format("delta") \
    .mode("overwrite") \
    .save(delta_path)

print(f"Датасет успішно збережено у Delta Lake за шляхом: {delta_path}")


Датасет успішно збережено у Delta Lake за шляхом: /Volumes/workspace/default/santader_delta_dataset
